# 1. Imports

In [1]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from aggregate_data import DataAggregator
from src.dim import Dims
from src.dicts import Dicts
import pandas as pd
import time

# 2. Variables

In [2]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DATAAGGREGATOR = DataAggregator(config, fetcher)
DIMS = Dims(config, fetcher)
DICTS = Dicts(config, fetcher)

25/10/21 20:29:31 WARN Utils: Your hostname, MacBook-Pro-Grzegorz.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
25/10/21 20:29:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/21 20:29:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/21 20:29:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 3. Data

## 3.1 Driver related data

In [8]:
driver_last_5_races_result = DATAAGGREGATOR.get_last_races_result(5,"Race", "position")
time.sleep(5)
driver_last_5_quali_result = DATAAGGREGATOR.get_last_races_result(5,"Qualifying", "position")
time.sleep(5)
driver_last_5_avg_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "avg")
time.sleep(5)
driver_last_5_std_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "std")
time.sleep(5)
driver_last_5_avg_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "avg")
time.sleep(5)
driver_last_5_std_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "std")
time.sleep(5)
driver_points_gathered = DATAAGGREGATOR.get_racer_team_points("driver")
time.sleep(10)
driver_gap_to_teammate = DATAAGGREGATOR.calculate_gap_to_teammate()
time.sleep(5)
driver_gap_to_leader = DATAAGGREGATOR.calculate_gap_to_leader()
dict_world_champions = DICTS.dict_world_champions()

/Users/grzegorznaporowski/Desktop/Portfolio/F1/aggregate_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_filtered["points_gap_to_teammate"] = merged_filtered["points_gained"] - merged_filtered["points_gained_other"]


## 3.2 Team related data

In [9]:
team_points_gathered = DATAAGGREGATOR.get_racer_team_points("team")

## 3.3 Session Related Data

In [10]:
dim_session = DIMS.dim_sessions()
dim_session = dim_session[dim_session["session_name"]=="Race"]
needed_cols = ["key", "location", "year", "is_current_season","country_name", "date_start"]
dim_session = dim_session[needed_cols]

# Joined Data

In [11]:

dim_session.merge(
    driver_last_5_races_result,
    on = "key",
    how="inner"
).merge(
    driver_last_5_quali_result,
    on = ["key","driver_number"],
    how = "inner",
    suffixes=["_race","_quali"]
).merge(
    driver_last_5_avg_race_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_std_race_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_avg_quali_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_std_quali_position,
    on = ["driver_number", "key"]
).merge(
    driver_points_gathered[["driver_number","key","points_gained"]],
    on = ["driver_number","key"]
).merge(
    driver_gap_to_teammate,
    on = ["driver_number","key"]
).merge(
    driver_gap_to_leader,
    on = ["driver_number","key"]
).merge(
    team_points_gathered,
    on = ["driver_number","key"]
).merge(
    dict_world_champions,
    on = "driver_number",
    how = "left"
)

,key,location,year,is_current_season,country_name,date_start,driver_number,position_race,last_race_pos_1,last_race_pos_2,...,avg_last_5_race,std_last_5_race,avg_last_5_qualifying,std_last_5_qualifying,points_gained,points_gap_to_teammate,gap_to_leader,team_name,team_points_gained,titles_count
0,Monaco2023,Monaco,2023,0,Monaco,2023-05-28,1,1.0,1.0,2.0,...,1.4,0.489898,5.6,5.571355,119.0,22.0,0.0,Red Bull Racing,216.0,4.0
1,Monaco2023,Monaco,2023,0,Monaco,2023-05-28,2,18.0,12.0,16.0,...,16.0,2.529822,18.0,2.280351,0.0,-1.0,119.0,Williams,1.0,NaN
2,Monaco2023,Monaco,2023,0,Monaco,2023-05-28,4,9.0,17.0,17.0,...,13.2,4.749737,13.2,4.118252,12.0,7.0,107.0,McLaren,17.0,NaN
3,Monaco2023,Monaco,2023,0,Monaco,2023-05-28,10,7.0,9.0,9.0,...,10.6,2.416609,12.6,5.885576,14.0,-7.0,105.0,Alpine,35.0,NaN
4,Monaco2023,Monaco,2023,0,Monaco,2023-05-28,11,16.0,2.0,1.0,...,2.2,1.469694,5.6,7.735632,97.0,-22.0,22.0,Red Bull Racing,216.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,Marina Bay2025,Marina Bay,2025,1,Singapore,2025-10-05,44,8.0,7.0,12.0,...,10.8,5.491812,10.4,3.929377,111.0,-53.0,204.0,Ferrari,275.0,7.0
1100,Marina Bay2025,Marina Bay,2025,1,Singapore,2025-10-05,55,10.0,18.0,14.0,...,11.8,4.955805,10.4,4.630335,29.0,-41.0,286.0,Williams,99.0,NaN
1101,Marina Bay2025,Marina Bay,2025,1,Singapore,2025-10-05,63,1.0,5.0,3.0,...,3.8,1.166190,5.2,0.748331,227.0,143.0,88.0,Mercedes,311.0,NaN
1102,Marina Bay2025,Marina Bay,2025,1,Singapore,2025-10-05,81,4.0,1.0,2.0,...,5.6,7.735632,3.4,2.870540,315.0,16.0,0.0,McLaren,614.0,NaN


In [10]:
dim_session

,key,location,year,is_current_season,country_name
1,Sakhir2023,Sakhir,2023,0,Bahrain
3,Jeddah2023,Jeddah,2023,0,Saudi Arabia
5,Melbourne2023,Melbourne,2023,0,Australia
9,Baku2023,Baku,2023,0,Azerbaijan
11,Miami2023,Miami,2023,0,United States
...,...,...,...,...,...
151,Zandvoort2025,Zandvoort,2025,1,Netherlands
153,Monza2025,Monza,2025,1,Italy
155,Baku2025,Baku,2025,1,Azerbaijan
157,Marina Bay2025,Marina Bay,2025,1,Singapore
